In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
#from sentence_transformers import SentenceTransformer
from sagemaker.pytorch import PyTorch
import os

In [2]:
# AWS params
bucket = "sagemaker-gpl"
dataset = "simple"
data_dir = f"generated/{dataset}"
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [ ]:
# Upload data to S3
def uploadDirectory(path, bucket):
    for root, dirs, files in os.walk(path):
        for file in files:
            if not file.startswith('.'):
                file_path = os.path.join(root, file)
                boto3.resource("s3").Bucket(bucket).upload_file(file_path, file_path)

# uploadDirectory(data_dir, bucket)

In [3]:
hyperparameters = {
#    "path_to_generated_data": f"generated/{dataset}",
    "base_ckpt": 'GPL/msmarco-distilbert-margin-mse',  
    "gpl_score_function": "dot",
    "batch_size_gpl": 32,
    "gpl_steps": 140000,
    "new_size": -1,
    "queries_per_passage": -1,
    "output_dir": f"/opt/ml/output/{dataset}",
    # "evaluation_data": f"./{dataset}",
    # "evaluation_output": f"evaluation/{dataset}",
    "generator": "BeIR/query-gen-msmarco-t5-base-v1",
    # "retrievers": ["msmarco-distilbert-base-v3", "msmarco-MiniLM-L-6-v3"],
    # "retriever_score_functions": ["cos_sim", "cos_sim"],
    "cross_encoder": "cross-encoder/ms-marco-MiniLM-L-6-v2",
    "qgen_prefix": "qgen"
}

In [4]:
trainer = PyTorch(
    role = role, 
    entry_point ='train.py',
    instance_type="ml.m5.large",
    instance_count=1,
    source_dir = './gpl',
    framework_version = '1.9.0',
    py_version = 'py38',
    sagemaker_session=sagemaker_session,
#    output_path=f"s3://{bucket}/model/embeddings",
    hyperparameters=hyperparameters,
)

In [8]:
# Take the contents of local folder path and put contents in folder key_prefix in bucket
# Documentation:
# https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session.upload_data
inputs = sagemaker_session.upload_data(path='generated', bucket=bucket, key_prefix='generated')

In [9]:
trainer.fit({'train': f"s3://{bucket}/{data_dir}"})

2022-09-21 08:39:40 Starting - Starting the training job...
2022-09-21 08:40:04 Starting - Preparing the instances for trainingProfilerReport-1663749580: InProgress
.........
2022-09-21 08:41:24 Downloading - Downloading input data......
2022-09-21 08:42:31 Training - Downloading the training image...
2022-09-21 08:43:06 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-09-21 08:43:09,124 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-09-21 08:43:09,128 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-09-21 08:43:09,146 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-09-21 08:43:09,155 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-09-21 08:43:09,700 sagemaker-training-toolkit INFO     I

KeyboardInterrupt: 